# LSTM et GRU - Les RNN Améliorés 🚀

## 🎯 Objectifs de ce notebook

1. **Comprendre** pourquoi les RNN basiques ont des limites
2. **Découvrir** comment LSTM et GRU résolvent ces problèmes
3. **Implémenter** et comparer RNN, LSTM et GRU
4. **Analyser** les performances sur des tâches complexes
5. **Choisir** le bon modèle selon le contexte

---

## 🤔 Rappel : Le problème des RNN

Dans le notebook précédent, nous avons vu que les RNN basiques ont un gros problème :

**Ils oublient rapidement** ! 🧠💨

### Exemple concret :
```
Phrase : "Ce film était nul au début mais finalement génial"
         ↑                                        ↑
      "nul"                                  "génial"
      
RNN basique : Se souvient surtout de "génial" → Prédiction correcte ✅
Mais si c'était : "Ce film était génial au début mais finalement nul"
RNN basique : Se souvient surtout de "nul" → Prédiction correcte ✅

Problème : Et si l'information importante est au milieu d'une longue phrase ?
RNN basique : Oublie le début → Prédiction incorrecte ❌
```

## 📦 Installation et imports

In [ ]:
# Installation des bibliothèques
!pip install tensorflow numpy matplotlib scikit-learn

In [ ]:
# Imports nécessaires
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import time

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ Imports terminés !")

## 🧠 LSTM : La Mémoire Sélective

### L'idée géniale des LSTM :

**"Et si on donnait au réseau le pouvoir de choisir quoi retenir et quoi oublier ?"**

### Les 3 "portes" magiques des LSTM :

1. **🚪 Porte d'oubli (Forget Gate)** : "Est-ce que je garde cette info ?"
2. **🚪 Porte d'entrée (Input Gate)** : "Est-ce que cette nouvelle info est importante ?"
3. **🚪 Porte de sortie (Output Gate)** : "Qu'est-ce que je montre à l'extérieur ?"

### Analogie simple :
Imaginez votre cerveau en train de lire :
- **Porte d'oubli** : "Cette info n'est plus utile, je l'oublie"
- **Porte d'entrée** : "Cette nouvelle info est importante, je la retiens"
- **Porte de sortie** : "Voici ce que je pense maintenant"

In [ ]:
# Visualisation simple du fonctionnement LSTM
def visualiser_lstm():
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    # Titre
    ax.text(0.5, 0.95, '🧠 Comment fonctionne un LSTM', 
            ha='center', va='top', fontsize=16, fontweight='bold', transform=ax.transAxes)
    
    # Cellule LSTM centrale
    lstm_rect = plt.Rectangle((0.4, 0.4), 0.2, 0.3, 
                             facecolor='lightblue', edgecolor='darkblue', linewidth=3)
    ax.add_patch(lstm_rect)
    ax.text(0.5, 0.55, 'LSTM\nCell', ha='center', va='center', 
            fontsize=12, fontweight='bold')
    
    # Les 3 portes
    gates = [
        {'pos': (0.3, 0.6), 'name': '🚪 Forget', 'color': 'red'},
        {'pos': (0.5, 0.75), 'name': '🚪 Input', 'color': 'green'},
        {'pos': (0.7, 0.6), 'name': '🚪 Output', 'color': 'orange'}
    ]
    
    for gate in gates:
        ax.text(gate['pos'][0], gate['pos'][1], gate['name'], 
                ha='center', va='center', fontsize=10, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor=gate['color'], alpha=0.7))
    
    # Flux d'information
    # Entrée
    ax.arrow(0.1, 0.55, 0.25, 0, head_width=0.03, head_length=0.02, 
             fc='blue', ec='blue', linewidth=2)
    ax.text(0.05, 0.55, '📝 Nouveau\nmot', ha='center', va='center', fontsize=10)
    
    # Mémoire précédente
    ax.arrow(0.5, 0.2, 0, 0.15, head_width=0.03, head_length=0.02, 
             fc='purple', ec='purple', linewidth=2)
    ax.text(0.5, 0.15, '🧠 Mémoire\nprécédente', ha='center', va='center', fontsize=10)
    
    # Sortie
    ax.arrow(0.65, 0.55, 0.25, 0, head_width=0.03, head_length=0.02, 
             fc='red', ec='red', linewidth=2)
    ax.text(0.95, 0.55, '🎯 Prédiction', ha='center', va='center', fontsize=10)
    
    # Nouvelle mémoire
    ax.arrow(0.5, 0.35, 0, -0.15, head_width=0.03, head_length=0.02, 
             fc='purple', ec='purple', linewidth=2)
    ax.text(0.5, 0.05, '🧠 Nouvelle\nmémoire', ha='center', va='center', fontsize=10)
    
    # Explications
    explanations = [
        '🚪 Forget: "Dois-je oublier l\'ancienne information ?"',
        '🚪 Input: "Cette nouvelle info est-elle importante ?"',
        '🚪 Output: "Que dois-je révéler maintenant ?"'
    ]
    
    for i, exp in enumerate(explanations):
        ax.text(0.02, 0.35 - i*0.05, exp, fontsize=9, transform=ax.transAxes)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

visualiser_lstm()

## 🔧 GRU : La Version Simplifiée

### Le problème des LSTM :
"3 portes, c'est peut-être un peu compliqué... 🤔"

### La solution GRU :
**"Faisons la même chose avec seulement 2 portes !"**

### Les 2 portes des GRU :
1. **🚪 Porte de remise à zéro (Reset Gate)** : "Dois-je oublier le passé ?"
2. **🚪 Porte de mise à jour (Update Gate)** : "Combien du passé et du présent je garde ?"

### Avantages des GRU :
- ✅ **Plus simple** que LSTM (2 portes au lieu de 3)
- ✅ **Plus rapide** à entraîner
- ✅ **Moins de paramètres**
- ✅ **Performances souvent similaires** aux LSTM

In [ ]:
# Comparaison visuelle des architectures
def comparer_architectures():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # RNN Basique
    ax1.set_title('🔴 RNN Basique', fontsize=14, fontweight='bold')
    rnn_rect = plt.Rectangle((0.3, 0.4), 0.4, 0.2, 
                            facecolor='lightcoral', edgecolor='darkred', linewidth=2)
    ax1.add_patch(rnn_rect)
    ax1.text(0.5, 0.5, 'RNN', ha='center', va='center', fontsize=12, fontweight='bold')
    ax1.text(0.5, 0.2, '• Pas de portes\n• Oublie rapidement\n• Simple mais limité', 
             ha='center', fontsize=9)
    
    # LSTM
    ax2.set_title('🔵 LSTM', fontsize=14, fontweight='bold')
    lstm_rect = plt.Rectangle((0.2, 0.4), 0.6, 0.2, 
                             facecolor='lightblue', edgecolor='darkblue', linewidth=2)
    ax2.add_patch(lstm_rect)
    ax2.text(0.5, 0.5, 'LSTM\n3 portes', ha='center', va='center', fontsize=12, fontweight='bold')
    ax2.text(0.5, 0.2, '• 3 portes\n• Excellente mémoire\n• Plus complexe', 
             ha='center', fontsize=9)
    
    # GRU
    ax3.set_title('🟢 GRU', fontsize=14, fontweight='bold')
    gru_rect = plt.Rectangle((0.25, 0.4), 0.5, 0.2, 
                            facecolor='lightgreen', edgecolor='darkgreen', linewidth=2)
    ax3.add_patch(gru_rect)
    ax3.text(0.5, 0.5, 'GRU\n2 portes', ha='center', va='center', fontsize=12, fontweight='bold')
    ax3.text(0.5, 0.2, '• 2 portes\n• Bon compromis\n• Plus rapide', 
             ha='center', fontsize=9)
    
    for ax in [ax1, ax2, ax3]:
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
    
    plt.suptitle('Comparaison des Architectures RNN', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

comparer_architectures()

## 🛠️ Implémentation : RNN vs LSTM vs GRU

Créons les trois modèles et comparons-les !

In [ ]:
# Fonctions pour créer les différents modèles
def creer_rnn_basique(vocab_size, embedding_dim, hidden_dim, max_length):
    """
    RNN basique (notre référence)
    """
    model = models.Sequential([
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        layers.SimpleRNN(hidden_dim, dropout=0.3),
        layers.Dense(1, activation='sigmoid')  # Classification binaire
    ])
    return model

def creer_lstm(vocab_size, embedding_dim, hidden_dim, max_length):
    """
    LSTM avec ses 3 portes magiques
    """
    model = models.Sequential([
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        layers.LSTM(hidden_dim, dropout=0.3),  # 🚪🚪🚪 3 portes
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def creer_gru(vocab_size, embedding_dim, hidden_dim, max_length):
    """
    GRU avec ses 2 portes simplifiées
    """
    model = models.Sequential([
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        layers.GRU(hidden_dim, dropout=0.3),    # 🚪🚪 2 portes
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Paramètres du modèle
VOCAB_SIZE = 1000
EMBEDDING_DIM = 50
HIDDEN_DIM = 64
MAX_LENGTH = 25  # Un peu plus long pour tester la mémoire

# Créer les trois modèles
modeles = {
    '🔴 RNN': creer_rnn_basique(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, MAX_LENGTH),
    '🔵 LSTM': creer_lstm(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, MAX_LENGTH),
    '🟢 GRU': creer_gru(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, MAX_LENGTH)
}

# Comparer le nombre de paramètres
print("📊 Comparaison du nombre de paramètres:")
print("=" * 45)
for nom, modele in modeles.items():
    nb_params = modele.count_params()
    print(f"{nom:8s}: {nb_params:,} paramètres")

print("\n💡 Observation:")
print("   LSTM > GRU > RNN en nombre de paramètres")
print("   Plus de paramètres = plus de complexité = plus de mémoire")

## 📊 Créons un Dataset Difficile

Pour vraiment tester les capacités de mémoire, créons des phrases où l'information importante est **au début** et le sentiment peut changer **à la fin** :

In [ ]:
# Dataset spécialement conçu pour tester la mémoire à long terme
def creer_dataset_difficile():
    """
    Crée des phrases où l'info importante est dispersée dans la phrase
    """
    
    # POSITIFS : Information importante à différents endroits
    positifs_complexes = [
        # Info importante au début, puis beaucoup de texte
        "Excellent ce produit même si le packaging pourrait être amélioré et la livraison a pris du temps mais au final je suis satisfait",
        "Fantastique cette expérience malgré quelques petits détails qui pourraient être perfectionnés selon moi mais c'est vraiment bien",
        "Parfait ce service bien que l'attente ait été longue et que le personnel semblait débordé mais le résultat est là",
        "Génial ce film même si certaines scènes sont un peu longues et l'histoire parfois prévisible mais j'ai adoré",
        "Superbe cette formation bien que certains concepts soient complexes et que le rythme soit parfois soutenu mais très enrichissant",
        
        # Info importante au milieu
        "Ce produit au début me semblait moyen mais finalement il est vraiment excellent et je le recommande vivement à tous",
        "L'expérience a commencé difficilement puis s'est révélée absolument fantastique et dépassé toutes mes attentes",
        "Le service paraissait décevant au premier contact mais s'est avéré parfait et très professionnel au final",
        
        # Info importante à la fin
        "Ce produit a un packaging ordinaire une présentation basique un prix élevé mais une qualité absolument exceptionnelle",
        "Cette formation a un contenu dense des exercices difficiles un rythme soutenu mais des résultats vraiment fantastiques"
    ]
    
    # NÉGATIFS : Information importante à différents endroits  
    negatifs_complexes = [
        # Info importante au début
        "Décevant ce produit même si l'emballage est joli et la présentation soignée et la livraison rapide mais la qualité n'y est pas",
        "Catastrophique cette expérience malgré un accueil chaleureux et un cadre agréable et des promesses alléchantes mais rien ne va",
        "Horrible ce service bien que le personnel soit souriant et les locaux propres et l'ambiance correcte mais le résultat est nul",
        "Nul ce film même si les acteurs sont connus et les décors beaux et la musique agréable mais l'histoire est ratée",
        "Mauvais cette formation bien que les supports soient beaux et la salle confortable et le formateur sympa mais inutile",
        
        # Info importante au milieu
        "Ce produit commençait bien avec de bonnes promesses mais s'est révélé vraiment décevant et ne vaut pas son prix du tout",
        "L'expérience a bien démarré avec un bon accueil puis s'est dégradée et est devenue catastrophique et frustrante",
        "Le service semblait professionnel au début mais s'est révélé horrible et décevant et vraiment pas à la hauteur",
        
        # Info importante à la fin  
        "Ce produit a un bel emballage une présentation soignée un marketing réussi mais une qualité vraiment décevante",
        "Cette formation a un programme alléchant des supports modernes un formateur expérimenté mais un contenu vraiment nul"
    ]
    
    # Phrases simples pour contrôle
    positifs_simples = [
        "Excellent produit parfait",
        "Très bon je recommande",
        "Fantastique vraiment bien",
        "Super expérience géniale",
        "Parfait très satisfait"
    ]
    
    negatifs_simples = [
        "Mauvais produit décevant",
        "Très nul je déconseille",
        "Horrible vraiment mal",
        "Catastrophique expérience nulle",
        "Décevant pas satisfait"
    ]
    
    # Combiner toutes les données (plus de phrases complexes)
    tous_textes = (
        positifs_complexes * 4 +  # Plus de phrases complexes
        negatifs_complexes * 4 +
        positifs_simples * 2 +     # Moins de phrases simples
        negatifs_simples * 2
    )
    
    # Labels correspondants
    labels = (
        [1] * len(positifs_complexes) * 4 +
        [0] * len(negatifs_complexes) * 4 +
        [1] * len(positifs_simples) * 2 +
        [0] * len(negatifs_simples) * 2
    )
    
    return tous_textes, np.array(labels)

# Créer le dataset
textes, labels = creer_dataset_difficile()

# Mélanger les données
indices = np.random.permutation(len(textes))
textes = [textes[i] for i in indices]
labels = labels[indices]

print(f"📊 Dataset créé: {len(textes)} exemples")
print(f"📊 Distribution: {np.sum(labels)} positifs, {len(labels) - np.sum(labels)} négatifs")

print("\n📝 Exemples de phrases complexes:")
print("=" * 60)
for i in range(3):
    sentiment = "😊 POSITIF" if labels[i] == 1 else "😞 NÉGATIF"
    print(f"{sentiment}:")
    print(f"  '{textes[i][:80]}...'\n")

In [ ]:
# Préparation des données
print("🔧 Préparation des données...")

# Tokenisation
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(textes)

# Conversion en séquences
sequences = tokenizer.texts_to_sequences(textes)

# Statistiques sur les longueurs
longueurs = [len(seq) for seq in sequences]
print(f"📏 Longueur moyenne: {np.mean(longueurs):.1f} mots")
print(f"📏 Longueur max: {np.max(longueurs)} mots")
print(f"📏 90% des phrases ont moins de {np.percentile(longueurs, 90):.0f} mots")

# Padding
X = pad_sequences(sequences, maxlen=MAX_LENGTH)
y = labels

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n✅ Données prêtes !")
print(f"   Train: {X_train.shape[0]} exemples")
print(f"   Test: {X_test.shape[0]} exemples")
print(f"   Longueur des séquences: {X_train.shape[1]} mots")

## 🏁 Le Grand Test : RNN vs LSTM vs GRU

Maintenant, entraînons les trois modèles et voyons qui se débrouille le mieux avec nos phrases complexes !

In [ ]:
# Fonction pour entraîner et évaluer un modèle
def entrainer_et_evaluer(modele, nom, X_train, y_train, X_test, y_test):
    """
    Entraîne un modèle et retourne ses performances
    """
    print(f"\n🚀 Entraînement du modèle {nom}...")
    
    # Compilation
    modele.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Mesurer le temps d'entraînement
    start_time = time.time()
    
    # Entraînement
    historique = modele.fit(
        X_train, y_train,
        batch_size=32,
        epochs=8,  # Moins d'epochs pour la démo
        validation_split=0.2,
        verbose=0  # Pas d'affichage détaillé
    )
    
    temps_entrainement = time.time() - start_time
    
    # Évaluation
    test_loss, test_accuracy = modele.evaluate(X_test, y_test, verbose=0)
    
    print(f"   ✅ Terminé en {temps_entrainement:.1f}s")
    print(f"   📊 Précision: {test_accuracy:.1%}")
    
    return {
        'modele': modele,
        'historique': historique,
        'precision': test_accuracy,
        'temps': temps_entrainement,
        'nom': nom
    }

# Entraîner tous les modèles
resultats = {}

for nom, modele in modeles.items():
    resultat = entrainer_et_evaluer(modele, nom, X_train, y_train, X_test, y_test)
    resultats[nom] = resultat

print("\n🏆 RÉSULTATS FINAUX")
print("=" * 40)

# Trier par précision
resultats_tries = sorted(resultats.items(), key=lambda x: x[1]['precision'], reverse=True)

for i, (nom, resultat) in enumerate(resultats_tries):
    medaille = ["🥇", "🥈", "🥉"][i]
    print(f"{medaille} {nom}: {resultat['precision']:.1%} (en {resultat['temps']:.1f}s)")

In [ ]:
# Visualisation des résultats
def visualiser_resultats(resultats):
    noms = list(resultats.keys())
    precisions = [resultats[nom]['precision'] for nom in noms]
    temps = [resultats[nom]['temps'] for nom in noms]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Graphique des précisions
    couleurs = ['red', 'blue', 'green']
    barres1 = ax1.bar(noms, precisions, color=couleurs, alpha=0.7)
    ax1.set_ylabel('Précision')
    ax1.set_title('🎯 Précision sur phrases complexes', fontweight='bold')
    ax1.set_ylim(0, 1)
    
    # Ajouter les valeurs sur les barres
    for barre, precision in zip(barres1, precisions):
        ax1.text(barre.get_x() + barre.get_width()/2, barre.get_height() + 0.01, 
                f'{precision:.1%}', ha='center', fontweight='bold')
    
    # Graphique des temps
    barres2 = ax2.bar(noms, temps, color=couleurs, alpha=0.7)
    ax2.set_ylabel('Temps (secondes)')
    ax2.set_title('⏱️ Temps d\'entraînement', fontweight='bold')
    
    # Ajouter les valeurs sur les barres
    for barre, t in zip(barres2, temps):
        ax2.text(barre.get_x() + barre.get_width()/2, barre.get_height() + 0.5, 
                f'{t:.1f}s', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualiser_resultats(resultats)

## 🔍 Test sur des Phrases Spécifiques

Testons nos modèles sur des phrases où la mémoire à long terme est cruciale :

In [ ]:
# Fonction pour prédire le sentiment
def predire_sentiment(phrase, modele, tokenizer, max_length):
    sequence = tokenizer.texts_to_sequences([phrase])
    sequence_paddee = pad_sequences(sequence, maxlen=max_length)
    score = modele.predict(sequence_paddee, verbose=0)[0, 0]
    
    if score > 0.5:
        return "😊 POSITIF", score
    else:
        return "😞 NÉGATIF", 1 - score

# Phrases test pour évaluer la mémoire
phrases_test = [
    # Phrase où l'info importante est au début
    "Excellent ce produit même si l'emballage pourrait être amélioré et que la livraison a pris du temps",
    
    # Phrase où l'info importante est à la fin
    "Ce produit a un emballage ordinaire une présentation basique mais une qualité absolument exceptionnelle",
    
    # Phrase avec négation au milieu
    "Au début ce film semblait ennuyeux mais finalement c'était vraiment fantastique",
    
    # Phrase courte pour comparaison
    "Film fantastique excellent",
    
    # Phrase négative complexe
    "Décevant ce produit malgré un bel emballage et de bonnes promesses marketing mais qualité nulle"
]

# Vrais labels (pour vérification)
vrais_labels = [1, 1, 1, 1, 0]  # 1=positif, 0=négatif

print("🧪 TEST SUR PHRASES COMPLEXES")
print("=" * 60)

for i, phrase in enumerate(phrases_test):
    vrai_sentiment = "😊 POSITIF" if vrais_labels[i] == 1 else "😞 NÉGATIF"
    
    print(f"\n📝 Phrase {i+1}: '{phrase[:50]}...'")
    print(f"   Vrai sentiment: {vrai_sentiment}")
    print("   Prédictions:")
    
    for nom, resultat in resultats.items():
        modele = resultat['modele']
        sentiment_pred, confidence = predire_sentiment(phrase, modele, tokenizer, MAX_LENGTH)
        
        # Vérifier si c'est correct
        correct = "✅" if (sentiment_pred == "😊 POSITIF" and vrais_labels[i] == 1) or \
                        (sentiment_pred == "😞 NÉGATIF" and vrais_labels[i] == 0) else "❌"
        
        print(f"     {nom}: {sentiment_pred} ({confidence:.1%}) {correct}")

## 📈 Analyse Avancée : Évolution pendant l'Entraînement

In [ ]:
# Visualiser l'évolution de l'entraînement
def visualiser_entrainement(resultats):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    couleurs = {'🔴 RNN': 'red', '🔵 LSTM': 'blue', '🟢 GRU': 'green'}
    
    for nom, resultat in resultats.items():
        historique = resultat['historique']
        couleur = couleurs[nom]
        
        # Accuracy
        ax1.plot(historique.history['accuracy'], label=f'{nom} (train)', 
                color=couleur, linewidth=2)
        ax1.plot(historique.history['val_accuracy'], label=f'{nom} (val)', 
                color=couleur, linewidth=2, linestyle='--')
    
    ax1.set_title('📈 Évolution de la Précision', fontweight='bold')
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Précision')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    for nom, resultat in resultats.items():
        historique = resultat['historique']
        couleur = couleurs[nom]
        
        # Loss
        ax2.plot(historique.history['loss'], label=f'{nom} (train)', 
                color=couleur, linewidth=2)
        ax2.plot(historique.history['val_loss'], label=f'{nom} (val)', 
                color=couleur, linewidth=2, linestyle='--')
    
    ax2.set_title('📉 Évolution de la Loss', fontweight='bold')
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualiser_entrainement(resultats)

## 🎯 Quand Utiliser Quoi ?

### Guide Pratique de Choix :

In [ ]:
# Tableau de recommandations
def afficher_guide_choix():
    print("🎯 GUIDE DE CHOIX : RNN vs LSTM vs GRU")
    print("=" * 60)
    
    situations = [
        {
            'situation': "🔤 Phrases courtes (< 10 mots)",
            'recommandation': "RNN basique suffit",
            'raison': "Pas besoin de mémoire complexe"
        },
        {
            'situation': "📚 Textes longs (> 20 mots)", 
            'recommandation': "LSTM ou GRU",
            'raison': "Mémoire à long terme nécessaire"
        },
        {
            'situation': "⚡ Besoin de rapidité",
            'recommandation': "GRU",
            'raison': "Plus rapide que LSTM, performances similaires"
        },
        {
            'situation': "🎯 Maximum de précision",
            'recommandation': "LSTM",
            'raison': "Souvent légèrement meilleur que GRU"
        },
        {
            'situation': "💾 Mémoire limitée",
            'recommandation': "GRU",
            'raison': "Moins de paramètres que LSTM"
        },
        {
            'situation': "🔄 Traduction/Seq2Seq",
            'recommandation': "LSTM",
            'raison': "Excelle dans les tâches séquence-à-séquence"
        }
    ]
    
    for situation in situations:
        print(f"\n{situation['situation']}")
        print(f"  → {situation['recommandation']}")
        print(f"  💡 {situation['raison']}")
    
    print("\n" + "=" * 60)
    print("🏆 RÈGLE D'OR : Commencez par GRU, puis testez LSTM si besoin !")

afficher_guide_choix()

## 🚀 Bonus : LSTM Bidirectionnel

**L'idée** : Et si on lisait la phrase dans les deux sens ?
- 📖 Un LSTM lit de gauche à droite
- 📖 Un autre LSTM lit de droite à gauche  
- 🧠 On combine les deux pour avoir une vision complète !

In [ ]:
# LSTM Bidirectionnel
def creer_lstm_bidirectionnel(vocab_size, embedding_dim, hidden_dim, max_length):
    """
    LSTM qui lit dans les deux sens
    """
    model = models.Sequential([
        layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        layers.Bidirectional(layers.LSTM(hidden_dim, dropout=0.3)),  # 🔄 Bidirectionnel !
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Créer et tester le modèle bidirectionnel
print("🔄 Test du LSTM Bidirectionnel...")

lstm_bidirectionnel = creer_lstm_bidirectionnel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, MAX_LENGTH)

# Comparer le nombre de paramètres
nb_params_normal = resultats['🔵 LSTM']['modele'].count_params()
nb_params_bidirect = lstm_bidirectionnel.count_params()

print(f"📊 LSTM normal:        {nb_params_normal:,} paramètres")
print(f"📊 LSTM bidirectionnel: {nb_params_bidirect:,} paramètres")
print(f"📊 Augmentation:       {(nb_params_bidirect/nb_params_normal - 1)*100:.0f}%")

# Entraîner rapidement
resultat_bidirect = entrainer_et_evaluer(
    lstm_bidirectionnel, "🔄 LSTM Bidirectionnel", 
    X_train, y_train, X_test, y_test
)

print(f"\n🏆 Comparaison finale:")
print(f"   🔵 LSTM normal:        {resultats['🔵 LSTM']['precision']:.1%}")
print(f"   🔄 LSTM bidirectionnel: {resultat_bidirect['precision']:.1%}")

if resultat_bidirect['precision'] > resultats['🔵 LSTM']['precision']:
    print("   💡 Le bidirectionnel gagne ! 🎉")
else:
    print("   💡 Pas d'amélioration significative cette fois.")

## 📋 Résumé et Conclusions

### 🏆 Ce que nous avons découvert :

1. **RNN basiques** 🔴 :
   - ✅ Simples et rapides
   - ❌ Oublient rapidement (problème du gradient qui disparaît)
   - 🎯 **Usage** : Phrases courtes, prototypage rapide

2. **LSTM** 🔵 :
   - ✅ Excellente mémoire à long terme (3 portes)
   - ✅ Résout le problème du gradient qui disparaît
   - ❌ Plus lent et plus complexe
   - 🎯 **Usage** : Textes longs, tâches complexes, traduction

3. **GRU** 🟢 :
   - ✅ Bon compromis performance/simplicité (2 portes)
   - ✅ Plus rapide que LSTM
   - ✅ Moins de paramètres
   - 🎯 **Usage** : Premier choix pour la plupart des tâches

### 💡 Conseils pratiques :

- **Débutant** : Commencez par GRU
- **Performance max** : Testez LSTM si GRU ne suffit pas
- **Vitesse importante** : Restez sur GRU
- **Phrases très longues** : LSTM bidirectionnel

### 🚀 Prochaines étapes :

1. **Attention Mechanisms** : Pour se concentrer sur les parties importantes
2. **Transformers** : L'architecture qui a révolutionné le NLP
3. **BERT, GPT** : Les modèles pré-entraînés modernes

---

### 🎉 Félicitations !

Vous maîtrisez maintenant :
- ✅ La différence entre RNN, LSTM et GRU
- ✅ Comment choisir le bon modèle
- ✅ L'implémentation avec TensorFlow
- ✅ L'évaluation sur des tâches complexes

**Vous êtes prêts pour les architectures encore plus avancées ! 🚀**